# Projet clustering

Objectif du projet : Utiliser le corpus export_articles_EGC_2004_2018 et faire le bilan de l'évolution de la communauté EGC ces 20 dernières années et tenter d'en prédire l'avenir.

##  Notes :
J'ai abandonné l'utilisation de TreeTagger pour lemmatiser : j'ai pas réussi à le faire fonctionner. Remplacé par FrenchLefffLemmatizer.

Ainsi que Gensim : la forme de ses corpus est étrange et je pense que c'est une perte de temps à se conformer à leur modèle. Remplacé par SKLearn.

### Imports

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from random import randint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.datasets import load_files
from sklearn.cluster import AgglomerativeClustering
import time

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
start_time = time.process_time()

### Ouverture du fichier de données

#### Fichier de données original.
doc = pd.read_csv("export_articles_EGC_2004_2018.csv", sep='\t')
doc

In [4]:
#### Fichier de données corrigé.
##### La correction va jusqu'à la ligne 112.
doc = pd.read_csv("export_articles_EGC_2004_2018_Correct.csv", sep='\t')
doc

,Unnamed: 0,series,booktitle,year,title,abstract,authors,pdf1page,pdfarticle
0,0,Revue des Nouvelles Technologies de l'Information,EGC,2018,#Idéo2017 : une plateforme citoyenne dédiée à ...,Cette plateforme a pour objectif de permettre ...,"Claudia Marinica, Julien Longhi, Nader Hassine...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002425
1,1,Revue des Nouvelles Technologies de l'Information,EGC,2018,A two level co-clustering algorithm for very l...,La classification croisée (co-clustering) est ...,"Marius Barctus, Marc Boullé, Fabrice Clérot",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002372
2,2,Revue des Nouvelles Technologies de l'Information,EGC,2018,ALGeoSPF: Un modèle de factorisation basé sur ...,La recommandation de points d'intérêts est dev...,"Jean-Benoît Griesner, Talel Abdesssalem, Huber...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002380
3,3,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse des sentiments à partir des commentair...,L'analyse des sentiments est un processus pend...,"Abdeljalil Elouardighi, Mohcine Maghfour, Hafd...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002397
4,4,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse en rôles sémantiques pour le résumé au...,Cet article présente une approche visant à ext...,"Elyase Lassouli, Yasmine Mesbahi, Camille Prad...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002384
5,5,Revue des Nouvelles Technologies de l'Information,EGC,2018,Analyse Ontologique de scénario dans un contex...,NaN,"Marwan Batrouni, Aurélie Bertaux, Christophe N...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002414
6,6,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apport de la fouille de données pour la préven...,Avec plus de 800 000 décès par an dans le mond...,"Romain Billot, Sofian Berrouiguet, Mark Larsen...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002376
7,7,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apport des modèles locaux pour les K-moyennes ...,"Dans le cadre du clustering prédictif, pour at...","Vincent Lemaire, Oumaima Alaoui Ismaili",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002379
8,8,Revue des Nouvelles Technologies de l'Information,EGC,2018,Apprendre les relations de préférence et de co...,"En classification multi-labels, chaque instanc...","Khalil Laghmari, Christophe Marsala, Mohammed ...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002381
9,9,Revue des Nouvelles Technologies de l'Information,EGC,2018,Approche contextuelle par régression pour les ...,Les tests A/B sont des procédures utilisées pa...,"Emmanuelle Claeys, Pierre Gançarski, Myriam Ma...",http://editions-rnti.fr/render_pdf.php?p1&p=10...,http://editions-rnti.fr/render_pdf.php?p=1002387


En travaillant avec title et abstract, on va réaliser les transformations.
Tous les documents ont des titres mais ils n'ont pas tous d'abstract.
On va donc utiliser une concaténation du titre et de l'abstract afin d'être sûr d'avoir de l'information.

### Lemmatisation et nettoyage des sujets

Il faut mettre les textes en minuscules, tokeniser, retirer la ponctuation, retirer les stopwords puis lemmatiser.

In [5]:
lemmer = FrenchLefffLemmatizer()

def lemm_tokens(tokens, lemmer):
    lemmed = []
    for item in tokens:
        lemmed.append(lemmer.lemmatize(item))
    return lemmed

def tokenize(text):
    text = text.translate(str.maketrans('','',string.punctuation))
    print(text)
    tokens = nltk.word_tokenize(text)
    lemms = lemm_tokens(tokens,lemmer)
    #stems = stem_tokens(tokens, stemmer)
    return lemms

In [6]:
resume = [tokenize(doc["title"][0].lower())]
abstract = [tokenize(str(doc["abstract"][0].lower()))]
for line in range(len(doc)): # title et abstract ont la même taille.
    if line != 0:
        resume.append(tokenize(doc["title"][line].lower()))
        abstract.append(tokenize(str(doc["abstract"][line]).lower()))

idéo2017  une plateforme citoyenne dédiée à lanalyse des tweets lors des événements politiques
cette plateforme a pour objectif de permettre aux citoyens danalyser par euxmêmes les tweets politiques lors dévénements spécifiques en france pour le cas de lélection présidentielle de 2017 idéo2017 analysait en quasi temps réel les messages des candidats et fournissait leurs principales caractéristiques lusage du lexique politique et des comparaisons entre les candidats
a two level coclustering algorithm for very large data sets
la classification croisée coclustering est une technique qui permet dextraire la structure sousjacente existante entre les lignes et les colonnes dune table de données sous forme de blocs plusieurs applications utilisent cette technique cependant de nombreux algorithmes de coclustering actuels ne passent pas à léchelle une des approches utilisées avec succès est la méthode modl qui optimise un critère de vraisemblance régularisée cependant pour des tailles plus impo

le clustering collaboratif cc vise à faire ressortir les structures communes présentes dans plusieurs vues indépendantes en se basant sur une première étape de clustering locale effectuée dans notre cas à laide de cartes autoorganisatrices som pour self organizing maps en anglais pour faire face à la quantité toujours croissante de données disponibles lutilisation de méthodes de clustering incrémentales est devenue nécessaire ce papier présente un algorithme de som incrémentales compatibles avec les contraintes du cc les expérimentations conduites sur plusieurs jeux de données démontrent la validité de cette méthode et présentent linfluence de la taille du batch utilisé lors de lapprentissage
catégorisation darticles scientifiques basée sur les relations sémantiques des motsclés
nan
classification de données complexes par globalisation de mesures de similarité via les moyennes quasiarithmétiques
la plupart des méthodes de classification sont conçues pour des types particuliers de donné

les systèmes de recommandation ont pour rôle daider les utilisateurs submergés par la quantité dinformation à faire de bons choix à partir de vastes catalogues de produits le déploiement de ces systèmes dans lindustrie hôtelière est confronté à des contraintes spécifiques limitant la performance des approches traditionnelles les systèmes de recommandation dhôtels souffrent en particulier dun problème de démarrage à froid continu à cause de la volatilité des préférences des voyageurs et du changement de comportements en fonction du contexte dans cet article nous présentons le problème de recommandation dhôtels ainsi que ses caractéristiques distinctives nous proposons de nouvelles méthodes contextuelles qui prennent en compte les dimensions géographique et temporelle ainsi que la raison du voyage afin de générer les listes de recommandation nos expérimentations sur des jeux de données réels soulignent la contribution des données contextuelles à lamélioration de la qualité de recommandat

anonymiser des données multidimensionnelles à laide du coclustering
dans cet article nous proposons une méthodologie pour anonymiser une table de données multidimensionnelles contenant des données individuelles soit n individus décrits par m variables lobjectif est de publier une table anonyme construite à partir dune table initiale qui protège contre le risque de réidentification en dautres termes on ne doit pas pouvoir retrouver dans les données publiées un individu présent dans la table originale la solution proposée consite à agréger les données à laide dune technique de coclustering puis à utiliser le modèle produit pour générer une table de données synthétiques du même format que les données initiales les données synthétiques qui contiennent des individus fictifs peuvent maintenant être publiées les données produites sont évaluées en termes dutilité pour différentes tâches de fouille analyse exploratoire classification et de niveau de protection
application du coclustering à lana

un grand nombre dapplications nécessitent danalyser un unique graphe attribué évoluant dans le temps cette tâche est particulièrement complexe car la structure du graphe et les attributs associés à chacun de ses noeuds ne sont pas figés dans ce travail nous nous focalisons sur la découverte de motifs récurrents dans un tel graphe ces motifs des séquences de sousgraphes connexes représentent les évolutions récurrentes de sousensembles de noeuds et de leurs attributs différentes contraintes ont été définies eg fréquence volume connectivité non redondance continuité et un algorithme original a été proposé les expérimentations réalisées sur des jeux de données synthétiques et réelles démontrent lintérêt de lapproche proposée et son passage à léchelle
extraction et chaînage supervisés de connaissances dun corpus dentretiens en histoire des sciences
nan
extraction et inférence de connaissances à partir dassemblages mécaniques définis par une représentation cao 3d
lextraction de connaissances

...

evolution de lontologie et gestion des annotations sémantiques inconsistantes
les ontologies et les annotations sémantiques sont deux composants importants dans un système de gestion des connaissances basé sur le web sémantique dans lenvironement dynamique et distribué du web sémantique les ontologies et les annotations pourraient être changées pour sadapter à lévolution de lorganisation concernée ces changements peuvent donc entraîner des inconsistances à détecter et traiter dans cet article nous nous focalisons principalement sur lévolution des annotations sémantiques en soulignant le contexte où les modifications de lontologie entraînent des inconsistances sur ces annotations nous présentons une approche basée sur des règles permettant de détecter les inconsistances dans les annotations sémantiques devenues obsolètes par rapport à lontologie modifiée nous décrivons aussi les stratégies dévolution nécessaires pour guider le processus de résolution de ces inconsistances grâce à des rè

avec le développement de compteurs communicants les consommations dénergie électrique pourront à terme être télérelevées par les fournisseurs délectricité à des pas de temps pouvant aller jusquà la seconde ceci générera des informations en continu à un rythme rapide et en quantité importante les systèmes de gestion de flux de données sgfd aujourdhui disponibles sous forme de prototypes ont vocation à faciliter la gestion de tels flux cette communication décrit une étude expérimentale pour analyser les avantages et limites de lutilisation de deux prototypes de sgfd stream et telegraphcq pour la gestion de données de consommation électrique
traitement et exploration du fichier log du serveur web pour lextraction des connaissances  web usage mining
le but dans ce travail consiste à concevoir et réaliser un outil logiciel en utilisant les concepts du web usage mining pour offrir aux web masters lensemble des connaissances y inclut les statistiques sur leurs sites afin de prendre les décisi

eda  algorithme de désuffixation du langage médical
nan
enrichissement dontologies dans le secteur de leau douce en environnement internet distribué et multilingue
nan
esiea datalab logiciel de nettoyage et préparation de données
nan
exploration des paramètres discriminants pour les représentations vectorielles de la sémantique des mots
les méthodes de représentation sémantique des mots à partir dune analyse statistique sont basées sur des comptes de cooccurences entre mots et unités textuelles ces méthodes ont des paramétrages complexes notamment le type dunité textuelle utilisée comme contexte ces paramètres déterminent fortement la qualité des résultats obtenus dans cet article nous nous intéressons au paramètrage de la technique dite hyperspace analogue to language halnous proposons une nouvelle méthode pour explorer ses paramètres discriminants cette méthode est basée sur lanalyse dun graphe de voisinage dune liste de mots de référence préclassés nous expérimentons cette méthode e

un problème important de la production automatique de règles de classification concerne la durée de génération de ces règles  en effet les algorithmes mis en œuvre produisent souvent des règles pendant un certain temps assez long nous proposons une nouvelle méthode de classification à partir dune base de données images cette méthode se situe à la jonction de deux techniques  lalgèbre de ptree et larbre de décision en vue daccélérer le processus de classification et de recherche dans de grandes bases dimages la modélisation que nous proposons se base dune part sur les descripteurs visuels tels que la couleur la forme et la texture dans le but dindexer les images et dautre part sur la génération automatique des règles de classification à laide dun nouvel algorithme c45ptree pour valider notre méthode nous avons développé un système baptisé ciadptree qui a été implémenté et confronté à une application réelle dans le domaine du traitement dimages les résultats expérimentaux montrent que ce

cet article présente lutilisation dune technique de fouille de données pour aider à la spécification de vues sur des sources xml notre langage de vues permet dintégrer des données xml provenant de sources hétérogènes cependant la définition de motifs sur les sources permettant de spécifier les données à extraire est souvent difficile car la structure des données nest pas toujours connue nous proposons dextraire les structures fréquentes dans les données des sources pour spécifier des motifs pertinents à utiliser dans la spécification des vues
détection par svm  application à la détection de churn en téléphonie mobile prépayée
nan
etiq un étiqueteur inductif convivail pour les corpus de spécialité
nan
étude de textes par leur image
nous proposons une méthode automatique de comparaison de textes reposant sur une technique de transformation dun texte en une image de taille donnée et lanalyse à laide des outils de la géométrie fractale nous présentons une application à létude dun corpus de

Le traitment des données est acceptable. Un meilleur traitement pourrait être envisagé.

Il faut obtenir un seul jeu de données avec les titres et les abstracts pour pouvoir les traiter et éliminer les cas qui n'ont pas d'abstract.

Il est possible que seul le titre suffise. Il est possible que la racinisation ne soit pas nécéssaire.

In [7]:
for line in range(len(abstract)):
    if abstract[line] != "nan":
        for element in abstract[line]:
            resume[line].append(element)

### Tf-Idf

Attention : l'étape suivante est longue.

In [8]:
dico_time = time.process_time()

In [9]:
dico = []
for line in resume:
    for word in line:
        dico.append(word)
for word in dico:
    if dico.count(word) != 1:
        dico.remove
len(dico)

140258

In [10]:
print("--- %s seconds ---" % round(time.process_time() - dico_time, 2))

--- 2103.98 seconds ---


In [11]:
dico = list(set(dico))
len(dico)

15079

In [12]:
# Calcul du Tf.
tf = []
for x in range(len(resume)):
    tf.append([])
    for y in range(len(resume[x])):
        tf[x].append(resume[x].count(resume[x][y]))

Attention : l'étape suivante est longue.

In [13]:
df_time = time.process_time()

In [14]:
# Calcul du Df.
df = {}
for x in range(len(dico)):
    sum = 0
    for y in range(len(resume)):
        if dico[x] in resume[y]:
            sum = sum + 1
    df[dico[x]] = sum

In [15]:
print("--- %s seconds ---" % round(time.process_time() - df_time, 2))

--- 186.41 seconds ---


In [16]:
# Calcul de l'Idf
idf = {}
for word in df:
    idf[word] = np.log10(len(resume)/df[word])

In [17]:
# Calcul du Tf-Idf.
tfidf = []
for x in range(len(resume)):
    tfidf.append({})
    for y in range(len(resume[x])):
        tfidf[x][resume[x][y]] = tf[x][y] * idf[resume[x][y]]
len(tfidf)

1269

Il faut trier les Tf-Idf par valeur et récupérer les 5 meilleurs.

In [18]:
best = []
for item in tfidf:
    topic = ""
    trier = sorted(item.items(), key=lambda kv: kv[1])
    trier.reverse()
    if len(item)<5:
        top = len(item)
    else:
        top = 5
    for i in range(top):
        topic = topic + trier[i][0] + " "
    best.append(topic)

In [19]:
best

['politique idéo2017 citoyen tweets candidat ',
 'coclustering modl gagne taille passent ',
 'recommandation mobilité factorisation dintérêts inféré ',
 'arabe sentiment commentaire marocain dialectal ',
 'cds résumé texte composant rôle ',
 'big scénario ontologique data analyse ',
 'tentative décès supervisées patient risque ',
 'formés majoritaire vote global obtenus ',
 'label heureux morceau relation musique ',
 'ab test procédure stratification contextuel ',
 'dynamics power understanding big human ',
 'incrémentales cc clustering som collaboratif ',
 'motsclés darticles catégorisation scientifique relation ',
 'cardiaque similarité quasiarithmétiques moyenne type ',
 'community complex network in structure ',
 'limportance noeud réseau influe différencie ',
 'similarité campagne sémantique vectorielles complémentarité ',
 'tempsréel oblige liot pertinence démontrées ',
 'complétude prescriptives commande contrainte vérifier ',
 'treillis distributif fca concept médians ',
 'gpo 

### Calcul du LSA

Le LSA va calculer, pour un document donné, les documents les plus similaires à son sujet. On classe le résultat le plus similaire dans le cluster du document donné. On reproduit cette étape et on arrête quand à un nombre de clusters ou une similarité trop faible.

En gros, le LSA nous fournit une similarité qu'on utilise pour former une classification hiérarchique.

On aura donc des clusters par sujets. On pourra élaborer nos techniques de datation pour déterminer les tendances et, si possible, créer de nouveaux clusters (avec les K-moyennes) plus précis.

In [20]:
## vectorisation
# Le tf-idf ne sert à rien d'autre que de convertir notre liste en un vecteur utilisable.
vectorizer = TfidfVectorizer(min_df =1)

dtm = vectorizer.fit_transform(best)

## Document Term Matrix vers Latent Sementic Analysis
lsa =TruncatedSVD(2, algorithm = 'randomized')

dtm_lsa = lsa.fit_transform(dtm)

dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

In [21]:
## Calcul de similarité des Documents

#Compute document similarity using LSA components

similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)

pd.DataFrame(similarity, index=best, columns=best).head(10)

,politique idéo2017 citoyen tweets candidat,coclustering modl gagne taille passent,recommandation mobilité factorisation dintérêts inféré,arabe sentiment commentaire marocain dialectal,cds résumé texte composant rôle,big scénario ontologique data analyse,tentative décès supervisées patient risque,formés majoritaire vote global obtenus,label heureux morceau relation musique,ab test procédure stratification contextuel,...,flou sousensembles appelons sousensemble défini,uitliation documentaire fondé contenu laide,nextclosure fermé itemsets partitionnement treillis,incomplets classement probabiliste dobjets arbre,galois dalgorithmes treillis supervisée étude,mask savoirfaire dappropriation dentreprise mémoire,voisinage graphe voisin proximité prétraitement,conceptuels graphe connaissance contrainte validation,veille technologique texte rechercher scientifique,naturel risque entrepôt dentrepôt examinons
politique idéo2017 citoyen tweets candidat,1.000000,0.657267,0.668790,0.866566,0.663839,0.967810,0.686585,0.703059,0.810150,0.658327,...,0.671391,0.672399,0.672567,0.668669,0.672624,0.663598,0.667256,0.674420,0.673066,0.671789
coclustering modl gagne taille passent,0.657267,1.000000,0.999882,0.193443,0.999962,0.446426,0.999216,0.998047,0.974296,0.999999,...,0.999821,0.999795,0.999790,0.999884,0.999789,0.999964,0.999911,0.999736,0.999776,0.999811
recommandation mobilité factorisation dintérêts inféré,0.668790,0.999882,1.000000,0.208522,0.999978,0.460148,0.999707,0.998891,0.977648,0.999902,...,0.999994,0.999988,0.999987,1.000000,0.999987,0.999976,0.999998,0.999971,0.999983,0.999992
arabe sentiment commentaire marocain dialectal,0.866566,0.193443,0.208522,1.000000,0.202024,0.964277,0.232128,0.254348,0.409488,0.194824,...,0.211948,0.213278,0.213499,0.208363,0.213575,0.201709,0.206506,0.215948,0.214159,0.212473
cds résumé texte composant rôle,0.663839,0.999962,0.999978,0.202024,1.000000,0.454242,0.999524,0.998556,0.976231,0.999973,...,0.999949,0.999934,0.999931,0.999979,0.999930,1.000000,0.999990,0.999899,0.999923,0.999943
big scénario ontologique data analyse,0.967810,0.446426,0.460148,0.964277,0.454242,1.000000,0.481497,0.501448,0.636529,0.447686,...,0.463256,0.464461,0.464662,0.460003,0.464730,0.453956,0.458316,0.466881,0.465260,0.463732
tentative décès supervisées patient risque,0.686585,0.999216,0.999707,0.232128,0.999524,0.481497,1.000000,0.999738,0.982450,0.999271,...,0.999786,0.999813,0.999817,0.999703,0.999819,0.999514,0.999655,0.999862,0.999830,0.999797
formés majoritaire vote global obtenus,0.703059,0.998047,0.998891,0.254348,0.998556,0.501448,0.999738,1.000000,0.986465,0.998134,...,0.999049,0.999108,0.999117,0.998883,0.999121,0.998539,0.998791,0.999220,0.999146,0.999073
label heureux morceau relation musique,0.810150,0.974296,0.977648,0.409488,0.976231,0.636529,0.982450,0.986465,1.000000,0.974612,...,0.978379,0.978659,0.978706,0.977614,0.978722,0.976161,0.977213,0.979218,0.978844,0.978490
ab test procédure stratification contextuel,0.658327,0.999999,0.999902,0.194824,0.999973,0.447686,0.999271,0.998134,0.974612,1.000000,...,0.999847,0.999822,0.999818,0.999904,0.999817,0.999975,0.999929,0.999767,0.999805,0.999838


### Clustering Hiérarchique Ascendant (HAC)

Le vrai point d'intérêt de cet algorithme est le seuil de similarité qui servira de point d'arrêt. Pour l'instant, je ne sais pas quel sera ce seuil, il faudra attendre les résultats du clustering.

In [22]:
clusters = AgglomerativeClustering(n_clusters=10).fit_predict(similarity)

In [23]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []

for i in range(len(clusters)):
    if clusters[i] == 0:
        c0.append({best[i]:doc["year"][i]})
    elif clusters[i] == 1:
        c1.append({best[i]:doc["year"][i]})
    elif clusters[i] == 2:
        c2.append({best[i]:doc["year"][i]})
    elif clusters[i] == 3:
        c3.append({best[i]:doc["year"][i]})
    elif clusters[i] == 4:
        c4.append({best[i]:doc["year"][i]})
    elif clusters[i] == 5:
        c5.append({best[i]:doc["year"][i]})
    elif clusters[i] == 6:
        c6.append({best[i]:doc["year"][i]})
    elif clusters[i] == 7:
        c7.append({best[i]:doc["year"][i]})
    elif clusters[i] == 8:
        c8.append({best[i]:doc["year"][i]})
    elif clusters[i] == 9:
        c9.append({best[i]:doc["year"][i]})

In [24]:
print(len(c0), len(c1), len(c2), len(c3), len(c4), len(c5), len(c6), len(c7), len(c8), len(c9))

75 21 4 73 13 19 31 2 1004 27


### Travail temporel

In [25]:
t0 = []
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
t6 = []
t7 = []
t8 = []
t9 = []

for i in range(len(c0)):
    t0.append(list(c0[i].values()))
for i in range(len(c1)):
    t1.append(list(c1[i].values()))
for i in range(len(c2)):
    t2.append(list(c2[i].values()))
for i in range(len(c3)):
    t3.append(list(c3[i].values()))
for i in range(len(c4)):
    t4.append(list(c4[i].values()))
for i in range(len(c5)):
    t5.append(list(c5[i].values()))
for i in range(len(c6)):
    t6.append(list(c6[i].values()))
for i in range(len(c7)):
    t7.append(list(c7[i].values()))
for i in range(len(c8)):
    t8.append(list(c8[i].values()))
for i in range(len(c9)):
    t9.append(list(c9[i].values()))   

In [26]:
for i in range(len(c0)):
    t0.append(t0[i][0])
t0 = t0[int(len(t0)/2):]
for i in range(len(c1)):
    t1.append(t1[i][0])
t1 = t1[int(len(t1)/2):]
for i in range(len(c2)):
    t2.append(t2[i][0])
t2 = t2[int(len(t2)/2):]
for i in range(len(c3)):
    t3.append(t3[i][0])
t3 = t3[int(len(t3)/2):]
for i in range(len(c4)):
    t4.append(t4[i][0])
t4 = t4[int(len(t4)/2):]
for i in range(len(c5)):
    t5.append(t5[i][0])
t5 = t5[int(len(t5)/2):]
for i in range(len(c6)):
    t6.append(t6[i][0])
t6 = t6[int(len(t6)/2):]
for i in range(len(c7)):
    t7.append(t7[i][0])
t7 = t7[int(len(t7)/2):]
for i in range(len(c8)):
    t8.append(t8[i][0])
t8 = t8[int(len(t8)/2):]
for i in range(len(c9)):
    t9.append(t9[i][0])
t9 = t9[int(len(t9)/2):]

In [27]:
T = [t0,t1,t2,t3,t4,t5,t6,t7,t8,t9]

In [28]:
def minYear(t):
    return np.min(t)

def maxYear(t):
    return np.max(t)

def moyYear(t):
    return int(np.mean(t))

In [29]:
recent = []
ancien = []
moyen = []
mort = []
nouveau = []

# Si ancien = récent alors récent
# Si récent = ancien alors ancien
# Si moyenne = ancien alors ancien
# Si moyenne = récent alors récent
# Si moyenne = moyen alors moyen

In [30]:
if minYear(t0) < 2011:
    if maxYear(t0) < 2011:
        mort.append(c0)
    else:
        if moyYear(t0) < 2009:
            ancien.append(c0)
        elif moyYear(t0) > 2014:
            recent.append(c0)
        else:
            moyen.append(c0)
elif minYear(t0) > 2011:
    nouveau.append(c0)
else:
    if moyYear(t0) < 2011:
        ancien.append(c0)
    elif moyYear(t0) > 2011:
        recent.append(c0)
    else:
        moyen.append(c0)
        
if minYear(t1) < 2011:
    if maxYear(t1) < 2011:
        mort.append(c1)
    else:
        if moyYear(t1) < 2009:
            ancien.append(c1)
        elif moyYear(t1) > 2014:
            recent.append(c1)
        else:
            moyen.append(c1)
elif minYear(t1) > 2011:
    nouveau.append(c1)
else:
    if moyYear(t1) < 2011:
        ancien.append(c1)
    elif moyYear(t1) > 2011:
        recent.append(c1)
    else:
        moyen.append(c1)  
        
if minYear(t2) < 2011:
    if maxYear(t2) < 2011:
        mort.append(c2)
    else:
        if moyYear(t2) < 2009:
            ancien.append(c2)
        elif moyYear(t2) > 2014:
            recent.append(c2)
        else:
            moyen.append(c2)
elif minYear(t2) > 2011:
    nouveau.append(c2)
else:
    if moyYear(t2) < 2011:
        ancien.append(c2)
    elif moyYear(t2) > 2011:
        recent.append(c2)
    else:
        moyen.append(c2)
                
if minYear(t3) < 2011:
    if maxYear(t3) < 2011:
        mort.append(c3)
    else:
        if moyYear(t3) < 2009:
            ancien.append(c3)
        elif moyYear(t3) > 2014:
            recent.append(c3)
        else:
            moyen.append(c3)
elif minYear(t3) > 2011:
    nouveau.append(c3)
else:
    if moyYear(t3) < 2011:
        ancien.append(c3)
    elif moyYear(t3) > 2011:
        recent.append(c3)
    else:
        moyen.append(c3)
                
if minYear(t4) < 2011:
    if maxYear(t4) < 2011:
        mort.append(c4)
    else:
        if moyYear(t4) < 2009:
            ancien.append(c4)
        elif moyYear(t4) > 2014:
            recent.append(c4)
        else:
            moyen.append(c4)
elif minYear(t4) > 2011:
    nouveau.append(c4)
else:
    if moyYear(t4) < 2011:
        ancien.append(c4)
    elif moyYear(t4) > 2011:
        recent.append(c4)
    else:
        moyen.append(c4)
                
if minYear(t5) < 2011:
    if maxYear(t5) < 2011:
        mort.append(c5)
    else:
        if moyYear(t5) < 2009:
            ancien.append(c5)
        elif moyYear(t5) > 2014:
            recent.append(c5)
        else:
            moyen.append(c5)
elif minYear(t5) > 2011:
    nouveau.append(c5)
else:
    if moyYear(t5) < 2011:
        ancien.append(c5)
    elif moyYear(t5) > 2011:
        recent.append(c5)
    else:
        moyen.append(c5)
                
if minYear(t6) < 2011:
    if maxYear(t6) < 2011:
        mort.append(c6)
    else:
        if moyYear(t6) < 2009:
            ancien.append(c6)
        elif moyYear(t6) > 2014:
            recent.append(c6)
        else:
            moyen.append(c6)
elif minYear(t6) > 2011:
    nouveau.append(c6)
else:
    if moyYear(t6) < 2011:
        ancien.append(c6)
    elif moyYear(t6) > 2011:
        recent.append(c6)
    else:
        moyen.append(c6)
                
if minYear(t7) < 2011:
    if maxYear(t7) < 2011:
        mort.append(c7)
    else:
        if moyYear(t7) < 2009:
            ancien.append(c7)
        elif moyYear(t7) > 2014:
            recent.append(c7)
        else:
            moyen.append(c7)
elif minYear(t7) > 2011:
    nouveau.append(c7)
else:
    if moyYear(t7) < 2011:
        ancien.append(c7)
    elif moyYear(t7) > 2011:
        recent.append(c7)
    else:
        moyen.append(c7)
                
if minYear(t8) < 2011:
    if maxYear(t8) < 2011:
        mort.append(c8)
    else:
        if moyYear(t8) < 2009:
            ancien.append(c8)
        elif moyYear(t8) > 2014:
            recent.append(c8)
        else:
            moyen.append(c8)
elif minYear(t8) > 2011:
    nouveau.append(c8)
else:
    if moyYear(t8) < 2011:
        ancien.append(c8)
    elif moyYear(t8) > 2011:
        recent.append(c8)
    else:
        moyen.append(c8)
                
if minYear(t9) < 2011:
    if maxYear(t9) < 2011:
        mort.append(c9)
    else:
        if moyYear(t9) <= 2010:
            ancien.append(c9)
        elif moyYear(t9) >= 2013:
            recent.append(c9)
        else:
            moyen.append(c9)
elif minYear(t9) > 2011:
    nouveau.append(c9)
else:
    if moyYear(t9) < 2011:
        ancien.append(c9)
    elif moyYear(t9) > 2011:
        recent.append(c9)
    else:
        moyen.append(c9)

In [31]:
print("Recent:",recent)
print("Ancien:",ancien)
print("Moyen:",moyen)
print("Mort:",mort)
print("Nouveau:",nouveau)

Recent: []
Ancien: []
Moyen: [[{'label heureux morceau relation musique ': 2018}, {'motsclés darticles catégorisation scientifique relation ': 2018}, {'’ d document interrogation schéma ': 2018}, {'blockchain notarisation nfb protocole document ': 2018}, {'indexation performante prise compte document ': 2018}, {'unitexgramlab grammaire sétend dunitexgramlab marnelavallée ': 2018}, {'thématique nouveauté émergentes dobserver scénario ': 2018}, {'label relation dapprendre dignorer cycliques ': 2017}, {'normalité singularité expression naturel langage ': 2017}, {'entité événement relation lexicosémantique rachetées ': 2017}, {'pharmacovigilance web social fondé sémantique ': 2017}, {'dopinions twitter recommandation plateforme danalyse ': 2017}, {'egc mot thématique topic 2016 ': 2016}, {'clustering contrainte satellite lexpert priori ': 2016}, {'protocole stockage libre cassandrales évaluépar ': 2016}, {'plateforme collaboratives qualité noeud relation ': 2016}, {'gstream flux clustering

In [32]:
print("--- %s seconds ---" % round(time.process_time() - start_time, 2))

--- 2317.86 seconds ---


### Évaluation de la classification

On va maximiser la distance inter cluster et minimiser la distance intra cluster

Distance intra max < Distance inter min

Comparons l'efficacité de la similarité de Jaccard et de la similarité Cosine

In [33]:
def jaccard(vec1, vec):
    commun = 0
    diff = 0
    for key in vec1.keys():
        if key in vec.keys():
            commun = commun + 1
        else:
            diff = diff + 1
    if diff == 0:
        return -1
    return commun/diff

def cosine(vec1, vec2):
    v1 = np.array(list(vec1.values()))
    v2 = np.array(list(vec2.values()))
    # La partie qui suit devra être retirée plus tard quand on aura des clusters normalisés.
    if len(vec1) > len(vec2):
        v1 = v1[:len(vec2)]
    else:
        v2 = v2[:len(vec1)]
    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))  

In [34]:
print("J",jaccard(c0,c1))
print("C",cosine(c0,c1))

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
print("--- %s seconds ---" % round(time.process_time() - start_time, 2))

### Post-soutenance

Réduire le vocabulaire.
Augmenter les clusters